In [1]:
import re
import os
import math
import torch
import pandas as pd

from tqdm import tqdm
from rouge import Rouge
from datetime import datetime

from tensorboardX import SummaryWriter
from transformers import AutoTokenizer
from torch.optim.lr_scheduler import _LRScheduler
from torch.utils.data import Dataset , DataLoader

from transformers import AutoTokenizer, BartForConditionalGeneration, BartConfig

/home/pervinco/miniconda3/envs/nlp-project/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

timestamp = datetime.now().strftime('%Y-%m-%d-%H-%M-%S')
save_path = os.path.join("./BART_runs", timestamp)

weights_path = os.path.join(save_path, 'weights')
logs_path = os.path.join(save_path, 'logs')
os.makedirs(weights_path, exist_ok=True)
os.makedirs(logs_path, exist_ok=True)

config = {
    "general": {
        "data_path": "../dataset",
        "train_file": "cleaned_train.csv",
        "valid_file": "cleaned_dev.csv",
        "test_file": "test.csv",
        "model_name": "gangyeolkim/kobart-korean-summarizer-v2",
        "output_dir": save_path,
        "model_cfg": ""
    },
    "tokenizer": {
        "path": "./tokenizer",
        "encoder_max_len": 512,
        "decoder_max_len": 256,
        "bos_token": "<s>",
        "eos_token": "</s>",
        "sep_token": "<sep>",
        "mask_token": "<mask>",
        "special_tokens": [
            "<sep>",
            '#Person1#',
            '#Person2#',
            '#Person3#',
            '#Person4#',
            '#Person5#',
            '#Person6#',
            '#Person7#',
            '#SSN#',
            '#Email#',
            '#Address#',
            '#Reaction#',
            '#CarNumber#',
            '#Movietitle#',
            '#DateOfBirth#',
            '#CardNumber#',
            '#PhoneNumber#',
            '#PassportNumber#'
        ]
    },
    "training": {
        "seed": 42,
        "overwrite_output_dir": True,

        "epochs": 1000,
        "batch_size": 32,
        "learning_rate": 0.000001,
        "weight_decay": 0.01,
        "accumulation_steps": 4,
        "patience": 10,

        "max_lr": 0.0001,
        "warmup_epochs": 10,
        "T_0": 100,
        "T_mult": 1,
        "T_gamma": 0.5,
    },
    "inference": {
        "no_repeat_ngram_size": 2,
        "early_stopping": False,
        "generate_max_length": 200,
        "num_beams": 4,
        "batch_size": 32,
        "remove_tokens": [
            "<unk>",
            "<s>",
            "</s>",
            "<pad>",
            "<sep>",
            "<mask>"
        ]
    }
}


In [3]:
bart_config = BartConfig.from_pretrained(config['general']['model_name'])

# tokenizer = BartTokenizerFast.from_pretrained("../tokenizer/ko_sentencepiece")
tokenizer = AutoTokenizer.from_pretrained(config['general']['model_name'], config=config)
remove_tokens = ['<usr>', f"{tokenizer.bos_token}", f"{tokenizer.unk_token}", f"{tokenizer.eos_token}", f"{tokenizer.pad_token}", f"{tokenizer.sep_token}", f"{tokenizer.mask_token}"]
tokenizer.add_special_tokens({"additional_special_tokens": config['tokenizer']['special_tokens']})
print(tokenizer.special_tokens_map)

model = BartForConditionalGeneration.from_pretrained(config['general']['model_name'])
model.resize_token_embeddings(len(tokenizer))
model.to(device)

optimizer = torch.optim.AdamW(model.parameters(), lr=config['training']['learning_rate'], weight_decay=config['training']['weight_decay'])

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


{'bos_token': '</s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>', 'mask_token': '<mask>', 'additional_special_tokens': ['<sep>', '#Person1#', '#Person2#', '#Person3#', '#Person4#', '#Person5#', '#Person6#', '#Person7#', '#SSN#', '#Email#', '#Address#', '#Reaction#', '#CarNumber#', '#Movietitle#', '#DateOfBirth#', '#CardNumber#', '#PhoneNumber#', '#PassportNumber#']}


You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
/home/pervinco/miniconda3/envs/nlp-project/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:615: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/home/pervinco/miniconda3/envs/nlp-project/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:634: UserWarning: `num_beams` is set to 1. However, `length_penalty` is set to `2.0` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `length_penalty`. This was detected when initializing 

In [4]:
train_df = pd.read_csv(f"{config['general']['data_path']}/{config['general']['train_file']}")
val_df = pd.read_csv(f"{config['general']['data_path']}/{config['general']['valid_file']}")
test_df = pd.read_csv(f"{config['general']['data_path']}/{config['general']['test_file']}")

In [5]:
class CustomDataset(Dataset):
    def __init__(self, df, tokenizer, is_train=True):
        self.tokenizer = tokenizer
        self.df = df.copy()
        self.is_train = is_train

        # 화자가 바뀔 때 SEP 토큰을 추가
        self.df.loc[:, 'dialogue'] = self.df['dialogue'].apply(self.add_sep_tokens)

        if self.is_train:           
            self.input_ids = tokenizer(self.df['dialogue'].tolist(), 
                                       return_tensors="pt", 
                                       padding=True,
                                       add_special_tokens=True, 
                                       truncation=True, 
                                       max_length=config['tokenizer']['encoder_max_len'], 
                                       return_token_type_ids=False).input_ids
            
            self.labels = tokenizer(self.df['summary'].tolist(), 
                                    return_tensors="pt", 
                                    padding=True,
                                    add_special_tokens=True, 
                                    truncation=True, 
                                    max_length=config['tokenizer']['decoder_max_len'],
                                    return_token_type_ids=False).input_ids
        else:
            self.input_ids = tokenizer(self.df['dialogue'].tolist(), 
                                       return_tensors="pt", 
                                       padding=True,
                                       add_special_tokens=True, 
                                       truncation=True, 
                                       max_length=config['tokenizer']['encoder_max_len'], 
                                       return_token_type_ids=False).input_ids

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        if self.is_train:
            return self.input_ids[idx], self.labels[idx]
        else:
            return self.input_ids[idx]


    def add_sep_tokens(self, dialogue):
        # 화자가 바뀔 때 SEP 토큰을 추가
        pattern = r'(#Person\d+#)'  # 화자를 나타내는 패턴
        parts = re.split(pattern, dialogue)  # 화자를 기준으로 대화 분리
        result = []
        prev_speaker = None
        for part in parts:
            if re.match(pattern, part):  # 화자가 바뀌면
                if prev_speaker and prev_speaker != part:
                    result.append('<sep>')  # SEP 토큰 추가
                prev_speaker = part
            result.append(part)
        return ''.join(result)

In [6]:
train_dataset = CustomDataset(train_df[['dialogue', 'summary']], tokenizer)
val_dataset = CustomDataset(val_df[['dialogue', 'summary']], tokenizer)
test_dataset = CustomDataset(test_df[['dialogue']], tokenizer, is_train=False)

train_loader = DataLoader(train_dataset, batch_size=config['training']['batch_size'], shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=config['training']['batch_size'], shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=config['training']['batch_size'], shuffle=False)

In [7]:
batch = next(iter(train_loader))
masked_input_ids, original_input_ids = batch[0], batch[1]

masked_example = tokenizer.decode(masked_input_ids[0], skip_special_tokens=False)
original_example = tokenizer.decode(original_input_ids[0], skip_special_tokens=False)

masked_example, original_example

('#Person1# 존이 헤어졌다는 소문이 있어. <sep>#Person2# 정말이야? 그런데 그의 아내는 항상 그를 매우 잘 돌봤는데. <sep>#Person1# 존이 다른 여자와 외도를 한다고 들었어. <sep>#Person2# 그의 아내 같은 여성을 속이다니 그는 정말 나쁜 사람이야.<pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><p

In [8]:
def ids_to_words(tokenizer, preds, labels):
    decoded_preds = tokenizer.batch_decode(preds, clean_up_tokenization_spaces=True)
    labels = tokenizer.batch_decode(labels, clean_up_tokenization_spaces=True)

    replaced_predictions = decoded_preds.copy()
    replaced_labels = labels.copy()

    for token in remove_tokens:
        replaced_predictions = [sentence.replace(token," ") for sentence in replaced_predictions]
        replaced_labels = [sentence.replace(token," ") for sentence in replaced_labels]
        
    return replaced_predictions, replaced_labels

In [9]:
def compute_metrics(replaced_predictions, replaced_labels):
    rouge = Rouge()

    results = rouge.get_scores(replaced_predictions, replaced_labels,avg=True)
    result = {key: value["f"] for key, value in results.items()}
    
    return result

In [10]:
class CosineAnnealingWarmUpRestarts(_LRScheduler):
    def __init__(self, optimizer, T_0, T_mult=1, eta_max=0.1, T_up=0, gamma=1., last_epoch=-1):
        if T_0 <= 0 or not isinstance(T_0, int):
            raise ValueError("Expected positive integer T_0, but got {}".format(T_0))
        if T_mult < 1 or not isinstance(T_mult, int):
            raise ValueError("Expected integer T_mult >= 1, but got {}".format(T_mult))
        if T_up < 0 or not isinstance(T_up, int):
            raise ValueError("Expected positive integer T_up, but got {}".format(T_up))
        self.T_0 = T_0
        self.T_mult = T_mult
        self.base_eta_max = eta_max
        self.eta_max = eta_max
        self.T_up = T_up
        self.T_i = T_0
        self.gamma = gamma
        self.cycle = 0
        self.T_cur = last_epoch
        super().__init__(optimizer, last_epoch)
    
    def get_lr(self):
        if self.T_cur == -1:
            return self.base_lrs
        elif self.T_cur < self.T_up:
            return [(self.eta_max - base_lr)*self.T_cur / self.T_up + base_lr for base_lr in self.base_lrs]
        else:
            return [base_lr + (self.eta_max - base_lr) * (1 + math.cos(math.pi * (self.T_cur-self.T_up) / (self.T_i - self.T_up))) / 2
                    for base_lr in self.base_lrs]

    def step(self, epoch=None):
        if epoch is None:
            epoch = self.last_epoch + 1
            self.T_cur = self.T_cur + 1
            if self.T_cur >= self.T_i:
                self.cycle += 1
                self.T_cur = self.T_cur - self.T_i
                self.T_i = (self.T_i - self.T_up) * self.T_mult + self.T_up
        else:
            if epoch >= self.T_0:
                if self.T_mult == 1:
                    self.T_cur = epoch % self.T_0
                    self.cycle = epoch // self.T_0
                else:
                    n = int(math.log((epoch / self.T_0 * (self.T_mult - 1) + 1), self.T_mult))
                    self.cycle = n
                    self.T_cur = epoch - self.T_0 * (self.T_mult ** n - 1) / (self.T_mult - 1)
                    self.T_i = self.T_0 * self.T_mult ** (n)
            else:
                self.T_i = self.T_0
                self.T_cur = epoch
                
        self.eta_max = self.base_eta_max * (self.gamma**self.cycle)
        self.last_epoch = math.floor(epoch)
        for param_group, lr in zip(self.optimizer.param_groups, self.get_lr()):
            param_group['lr'] = lr

In [11]:
def train(epoch, model, device, train_loader, optimizer, writer, accumulation_steps):
    model.train()
    total_loss = 0.0
    optimizer.zero_grad()

    for idx, batch in tqdm(enumerate(train_loader), total=len(train_loader), desc=f"Training Epoch {epoch}", leave=False):
        input_ids = batch[0].to(device, dtype=torch.long)
        labels = batch[1].to(device, dtype=torch.long)

        outputs = model(input_ids=input_ids, labels=labels)
        ce_loss = outputs.loss
        ce_loss = ce_loss / accumulation_steps 
        ce_loss.backward()

        if (idx + 1) % accumulation_steps == 0:
            optimizer.step()
            optimizer.zero_grad()

        total_loss += ce_loss.item() * accumulation_steps

    avg_loss = total_loss / len(train_loader)
    writer.add_scalar('Loss/train', avg_loss, epoch)

    return avg_loss

def validate(tokenizer, model, device, val_loader, writer, epoch):
    model.eval()
    total_loss = 0
    all_results = []
    all_predictions = []
    all_labels = []

    with torch.no_grad():
        for idx, batch in tqdm(enumerate(val_loader), total=len(val_loader), desc="Validating", leave=False):
            input_ids = batch[0].to(device, dtype=torch.long)
            labels = batch[1].to(device, dtype=torch.long)

            pred_ids = model.generate(
                input_ids=input_ids,
                max_length=config['inference']['generate_max_length'], 
                num_beams=config['inference']['num_beams'],
                repetition_penalty=2.0, 
                length_penalty=1.0, 
                early_stopping=True,
                no_repeat_ngram_size=2
            )

            loss = model(input_ids=input_ids, labels=labels).loss
            total_loss += loss.item()

            replaced_predictions, replaced_labels = ids_to_words(tokenizer, pred_ids, labels)
            result = compute_metrics(replaced_predictions, replaced_labels)

            all_results.append(result)
            all_predictions.extend(replaced_predictions)
            all_labels.extend(replaced_labels)

    val_loss = total_loss / len(val_loader)
    avg_result = {key: sum(r[key] for r in all_results) / len(all_results) for key in all_results[0]}
    
    writer.add_scalar('Loss/valid', val_loss, epoch)
    writer.add_scalar('ROUGE/rouge-1', avg_result['rouge-1'], epoch)
    writer.add_scalar('ROUGE/rouge-2', avg_result['rouge-2'], epoch)
    writer.add_scalar('ROUGE/rouge-l', avg_result['rouge-l'], epoch)

    return val_loss, avg_result, all_predictions, all_labels

In [12]:
best_rouge = 0
early_stopping_counter = 0
best_val_loss = float('inf')
writer = SummaryWriter(log_dir=logs_path)
scheduler = CosineAnnealingWarmUpRestarts(optimizer, T_0=config['training']['T_0'], T_mult=config['training']['T_mult'], eta_max=config['training']['max_lr'],  T_up=config['training']['warmup_epochs'], gamma=config['training']['T_gamma'])

for epoch in range(1, config['training']['epochs'] + 1):
    current_lr = optimizer.param_groups[0]['lr']
    print(f"Epoch {epoch}/{config['training']['epochs']}, Current Learning Rate: {current_lr:.6f}")
    train_loss = train(epoch, model, device, train_loader, optimizer, writer, config['training']['accumulation_steps'])
    val_loss, val_result, val_predictions, val_labels = validate(tokenizer, model, device, val_loader, writer, epoch)

    # avg_rouge = (val_result['rouge-1'] + val_result['rouge-2'] + val_result['rouge-l']) / 3
    print(f"Train Loss: {train_loss:.6f}, Valid Loss: {val_loss:.6f}")
    print(f"Rouge-1: {val_result['rouge-1']:.6f}, Rouge-2: {val_result['rouge-2']:.6f}, Rouge-l: {val_result['rouge-l']:.6f}")

    scheduler.step()
    print('-'*150)
    for i in range(3):
        print(f"PRED: {val_predictions[i].strip()}")
        print(f"GOLD: {val_labels[i]}")
        print('-'*150)

    # if avg_rouge > best_rouge:
    if val_result['rouge-1'] > best_rouge:
        # best_rouge = avg_rouge
        best_rouge = val_result['rouge-1']
        early_stopping_counter = 0 
        torch.save(model.state_dict(), os.path.join(weights_path, 'best_finetune.pth'))
        print(f"New best model saved with average ROUGE: {best_rouge:.6f}")

    else:
        early_stopping_counter += 1
        print(f"Not improve. Early stopping counter: {early_stopping_counter}/{config['training']['patience']}")

    if early_stopping_counter >= config['training']['patience']:
        print("Early stopping triggered.")
        break

    torch.save(model.state_dict(), os.path.join(weights_path, f'epoch-{epoch}_finetune.pth'))
    print()

writer.close()
torch.save(model.state_dict(), os.path.join(weights_path, 'last_finetune.pth'))
print("Training completed. Last model saved.")

Epoch 1/1000, Current Learning Rate: 0.000001


Validating:   0%|          | 0/16 [00:00<?, ?it/s]                 /home/pervinco/miniconda3/envs/nlp-project/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:615: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/home/pervinco/miniconda3/envs/nlp-project/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:634: UserWarning: `num_beams` is set to 1. However, `length_penalty` is set to `2.0` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `length_penalty`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect

Train Loss: 5.979798, Valid Loss: 4.671269
Rouge-1: 0.153446, Rouge-2: 0.034655, Rouge-l: 0.149603
------------------------------------------------------------------------------------------------------------------------------------------------------
PRED: 숨쉬기가 힘들고 감기에 걸리신 적이 있냐는 질문에 알고 있는 알레르기는 없다고 답했다. 그리고 폐 전문의에게 천식 검사를 받게 할 것이다.
GOLD: #Person2# 는 숨쉬기에 어려움을 겪는다. 의사는 #Person1# 에게 이에 대해 묻고, #Person2# 를 폐 전문의에게 보낼 예정이다.                                                             
------------------------------------------------------------------------------------------------------------------------------------------------------
PRED: 다리와 팔목을 운동하고 주간 스케줄을 따르고 있어 너 때문에 모든 게 망가지고 있어 금요일에 다리를 할 수 있어. 그럼 3시 30분에 헬스장에서 만나자.
GOLD: #Person1# 은 지미에게 운동하러 가자고 제안하고 팔과 배를 운동하도록 설득한다.                                                                       
---------------------------------------------------------------------------------------------------------------------------------------------------

Train Loss: 2.606532, Valid Loss: 1.183783
Rouge-1: 0.178458, Rouge-2: 0.043076, Rouge-l: 0.173609
------------------------------------------------------------------------------------------------------------------------------------------------------
PRED: 숨쉬기가 힘들고 감기에 걸리신 적이 있냐는 질문에 알레르기는 없다고 답했다. 그리고 폐 전문의에게 천식에 대한 검사를 받게 할 것이다.
GOLD: #Person2# 는 숨쉬기에 어려움을 겪는다. 의사는 #Person1# 에게 이에 대해 묻고, #Person2# 를 폐 전문의에게 보낼 예정이다.                                                             
------------------------------------------------------------------------------------------------------------------------------------------------------
PRED: 헤이, 지미는 다리와 팔목을 운동하고 금요일에 다리를 할 수 있다고 말한다. 3시 30분에 헬스장에서 만나자. 그 다음에는 팔과 배를 운동하자.
GOLD: #Person1# 은 지미에게 운동하러 가자고 제안하고 팔과 배를 운동하도록 설득한다.                                                                       
------------------------------------------------------------------------------------------------------------------------------------------------------
PRE

Train Loss: 0.599097, Valid Loss: 0.697122
Rouge-1: 0.211871, Rouge-2: 0.058043, Rouge-l: 0.205401
------------------------------------------------------------------------------------------------------------------------------------------------------
PRED: #Person2# 는 숨쉬기가 힘들고 알레르기가 있다. 폐 전문의에게 천식에 대한 검사를 받을 수 있도록 도와준 의사 선생님에게 감사합니다.
GOLD: #Person2# 는 숨쉬기에 어려움을 겪는다. 의사는 #Person1# 에게 이에 대해 묻고, #Person2# 를 폐 전문의에게 보낼 예정이다.                                                             
------------------------------------------------------------------------------------------------------------------------------------------------------
PRED: 헤이, 지미는 다리와 팔목을 운동하고 금요일에 다리를 할 수 있도록 헬스장에서 만나자고 제안한다. 헤이는 주간 스케줄을 따르고 있다.
GOLD: #Person1# 은 지미에게 운동하러 가자고 제안하고 팔과 배를 운동하도록 설득한다.                                                                       
------------------------------------------------------------------------------------------------------------------------------------------------------
PRED: 

...

Train Loss: 0.092766, Valid Loss: 0.671574
Rouge-1: 0.258295, Rouge-2: 0.080773, Rouge-l: 0.249416
------------------------------------------------------------------------------------------------------------------------------------------------------
PRED: #Person2# 는 숨쉬기가 힘들고 운동을 할 때 주로 나타나기 때문에 천식에 대한 검사를 받고 싶어합니다. 의사는 그를 폐 전문의에게 보내기로 결정했습니다.
GOLD: #Person2# 는 숨쉬기에 어려움을 겪는다. 의사는 #Person1# 에게 이에 대해 묻고, #Person2# 를 폐 전문의에게 보낼 예정이다.                                                             
------------------------------------------------------------------------------------------------------------------------------------------------------
PRED: #Person1# 은 지미에게 운동하자고 제안한다. 그들은 금요일에 다리와 팔목을 운동하기로 결정하고, 3시 30분에 헬스장에서 만날 예정이다.
GOLD: #Person1# 은 지미에게 운동하러 가자고 제안하고 팔과 배를 운동하도록 설득한다.                                                                       
--------------------------------------------------------------------------------------------------------------------------------------------

Train Loss: 0.083883, Valid Loss: 0.683925
Rouge-1: 0.260115, Rouge-2: 0.081548, Rouge-l: 0.251097
------------------------------------------------------------------------------------------------------------------------------------------------------
PRED: #Person2# 는 숨쉬기가 힘들고 운동을 할 때 주로 나타나기 때문에 천식에 대한 검사를 받게 될 거라고 의사에게 말했습니다. 의사는 그를 폐 전문의에게 보내기로 했습니다.
GOLD: #Person2# 는 숨쉬기에 어려움을 겪는다. 의사는 #Person1# 에게 이에 대해 묻고, #Person2# 를 폐 전문의에게 보낼 예정이다.                                                             
------------------------------------------------------------------------------------------------------------------------------------------------------
PRED: #Person1# 와 지미는 운동하기로 결정한다. 그들은 금요일에 다리와 팔목을 운동할 예정이다. 그들은 3시 30분에 헬스장에서 만나기로 한다.
GOLD: #Person1# 은 지미에게 운동하러 가자고 제안하고 팔과 배를 운동하도록 설득한다.                                                                       
----------------------------------------------------------------------------------------------------------------------------------

Train Loss: 0.075548, Valid Loss: 0.690186
Rouge-1: 0.259236, Rouge-2: 0.081130, Rouge-l: 0.251468
------------------------------------------------------------------------------------------------------------------------------------------------------
PRED: #Person2# 는 숨쉬기가 힘들고 운동을 할 때 주로 나타나기 때문에 천식에 대한 검사를 받게 될까봐 걱정한다. 의사는 그를 폐 전문의에게 보내기로 결정한다.
GOLD: #Person2# 는 숨쉬기에 어려움을 겪는다. 의사는 #Person1# 에게 이에 대해 묻고, #Person2# 를 폐 전문의에게 보낼 예정이다.                                                             
------------------------------------------------------------------------------------------------------------------------------------------------------
PRED: #Person1# 와 지미는 운동하기로 결정한다. 그들은 금요일에 다리와 팔목을 운동할 예정이다. 그들은 3시 30분에 헬스장에서 만나기로 한다.
GOLD: #Person1# 은 지미에게 운동하러 가자고 제안하고 팔과 배를 운동하도록 설득한다.                                                                       
------------------------------------------------------------------------------------------------------------------------------------------

Train Loss: 0.067626, Valid Loss: 0.707518
Rouge-1: 0.257548, Rouge-2: 0.080393, Rouge-l: 0.248544
------------------------------------------------------------------------------------------------------------------------------------------------------
PRED: #Person2# 는 최근에 숨쉬기가 힘들고, 운동을 할 때 주로 나타나기 때문에 천식에 대한 검사를 받게 될까봐 걱정하고 있습니다.. 의사는
GOLD: #Person2# 는 숨쉬기에 어려움을 겪는다. 의사는 #Person1# 에게 이에 대해 묻고, #Person2# 를 폐 전문의에게 보낼 예정이다.                                                             
------------------------------------------------------------------------------------------------------------------------------------------------------
PRED: #Person1# 와 지미는 운동 계획에 대해 이야기한다. 그들은 금요일에 다리와 팔목을 운동하기로 결정하고, 3시 30분에 헬스장에서 만날 예정이다.
GOLD: #Person1# 은 지미에게 운동하러 가자고 제안하고 팔과 배를 운동하도록 설득한다.                                                                       
--------------------------------------------------------------------------------------------------------------------------------------------------

Train Loss: 0.060660, Valid Loss: 0.718938
Rouge-1: 0.264583, Rouge-2: 0.084810, Rouge-l: 0.254733
------------------------------------------------------------------------------------------------------------------------------------------------------
PRED: #Person2# 는 숨쉬기가 힘들고, 알고 있는 알레르기는 없지만 운동을 할 때 많이 나타나기 때문에 천식에 대한 검사를 받게 될 거라고 말했습니다.
GOLD: #Person2# 는 숨쉬기에 어려움을 겪는다. 의사는 #Person1# 에게 이에 대해 묻고, #Person2# 를 폐 전문의에게 보낼 예정이다.                                                             
------------------------------------------------------------------------------------------------------------------------------------------------------
PRED: #Person1# 과 지미는 운동하기로 결정하고, 지미가 주간 스케줄을 따르고 있기 때문에 금요일에 다리를 하기로 한다. 그들은 헬스장에서 만날 예정이다.
GOLD: #Person1# 은 지미에게 운동하러 가자고 제안하고 팔과 배를 운동하도록 설득한다.                                                                       
-------------------------------------------------------------------------------------------------------------------------------------------

In [13]:
def predict(tokenizer, model, device, test_loader, fname):
    model.eval()
    summary = []
    with torch.no_grad():
        for input_ids in tqdm(test_loader):
            input_ids = input_ids.to(device, dtype=torch.long)

            pred_ids = model.generate(
                input_ids=input_ids,
                max_length=config['inference']['generate_max_length'], 
                num_beams=config['inference']['num_beams'],
                repetition_penalty=2.0, 
                length_penalty=1.0, 
                early_stopping=True,
                no_repeat_ngram_size=2
            )
            for ids in pred_ids:
                result = tokenizer.decode(ids)
                summary.append(result)
                
    # remove_tokens = ['<usr>', f"{tokenizer.unk_token}", f"{tokenizer.eos_token}", f"{tokenizer.pad_token}"]
    preprocessed_summary = summary.copy()
    for token in remove_tokens:
        preprocessed_summary = [sentence.replace(token," ") for sentence in preprocessed_summary]

    output = pd.DataFrame(
        {
            "fname": fname,
            "summary" : preprocessed_summary,
        }
    )
    return output

In [14]:
model.load_state_dict(torch.load(f'{save_path}/weights/best_finetune.pth'))
output = predict(tokenizer, model, device, test_loader, test_df['fname'])
output.to_csv(f"{save_path}/prediction.csv", index=False)

  0%|          | 0/16 [00:00<?, ?it/s]/home/pervinco/miniconda3/envs/nlp-project/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:615: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/home/pervinco/miniconda3/envs/nlp-project/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:634: UserWarning: `num_beams` is set to 1. However, `length_penalty` is set to `2.0` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `length_penalty`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should 